In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
import nltk
nltk.download('punkt')
from tokenizers import ByteLevelBPETokenizer
from transformers import AutoTokenizer

# Load the Pokemon descriptions dataset
pokemon_descriptions = pd.read_csv('/content/drive/MyDrive/pokemon/poki_data.csv')

In [ ]:
# Initialize the tokenizer
tokenizer = ByteLevelBPETokenizer("/content/drive/MyDrive/pokemon/vocab.json", "/content/drive/MyDrive/pokemon/merges.txt")

# Tokenize Pokémon names
pokemon_descriptions['name'] = pokemon_descriptions['name'].apply(lambda x: tokenizer.encode(x))


In [ ]:
# Initialize the tokenizer
description_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize Pokémon descriptions
pokemon_descriptions['desc'] = pokemon_descriptions['desc'].apply(lambda x: description_tokenizer.encode(x))


In [ ]:
import torch
from torch.utils.data import Dataset

class PokemonDataset(Dataset):
   def __init__(self, encodings, labels):
       self.encodings = encodings
       self.labels = labels

   def __getitem__(self, idx):
       item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
       item["labels"] = torch.tensor(self.labels[idx])
       return item

   def __len__(self):
       return len(self.labels)

# Convert the DataFrame to a list of dictionaries
inputs = pokemon_descriptions['name'].tolist()
targets = pokemon_descriptions['desc'].tolist()

# # Encode the inputs


# Create the dataset
dataset = PokemonDataset(inputs, targets)


In [ ]:
import pickle

# Save the dataset to a file using pickle
file_path = '/content/drive/MyDrive/pokemon/pokemon_dataset.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(dataset, file)

print(f"Dataset saved to {file_path}")
